# Artificial Intelligence
## UEMH3073 / UECS2053 / UECS2153

# Lab 2: Genetic Algorithm

This notebook is an assignment requiring you to investigate the Travelling Salesman Problem. Guidance is provided so you can understand what needs to be done for this assignment as you follow through this lab. Convenience classes and functions/ methods are provided.

You will encounter #TODO in the code cells explaining tasks you need to complete. In other words, you will need to write codes and accomplish the #TODO tasks so that the genetic algorithm functions well and runs correctly. Look for "Replacement starts here" and "Replacement ends here" to know the parts of the codes requiring your inputs.
    

The #TODO tasks and their marks distribution are as follows:
 
a. #TODO1 (10 marks) in the Population Initialization function. You will read a set of cities from the filename when creating an initial population. 

b. #TODO2 (10 marks) in the Parent Selection function. You will replace a dummy parent selection function with Tournament Selection. 

c. #TODO3 (10 marks) in the Parent Selection function. You will replace a dummy parent selection function with Proportional Selection.

d. #TODO4 (10 marks) in the Survival Selection function. You will replace the dummy survival selection function with Merge, Sort & Truncate. 
    
e. #TODO5 (10 marks) in the Crossover function. You will replace the dummy crossover function the Partially Mapped Crossover approach.

f. #TODO6 (10 marks) in the Mutation function. You will replace the dummy mutation function with Insertion Mutation approach. 

g. #TODO7 (10 marks) in Performance Evaluation. You will present performance evaluation for the different Parent Selection functions. 

Marks are also given for: Report Presentation and Formatting (15%) and Code Quality and Comments (15%). More details about this notebook and assignemnt are provided in your lab sheet.

## An Overview of the Travelling Salesman Problem

In the travelling salesman problem, a salesperson wish to find the shortest path that passes through all cities s/he wishes to visit given the coordinates of a set of cities. The salesperson should visit each of the cities once only, and so:

a. Each path consists all cities in the set.

b. Each path visits each of the cities once only. So, none of the cities are visited more than once. 

## Imports

In [2]:
%matplotlib inline

# Please add more imports if you need them 

import random
import time
import csv

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pprint import pprint as print 

## Convenience Classes

### City

The City class, which represents a city, possesses the properties of the city and has functions/ methods used for calculating the distance between the city and another city. Each path, represented by a chromosome, is formed by a set of cities.   

In [3]:
class City:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def distance(self, city):
        xDis = abs(self.x - city.x)
        yDis = abs(self.y - city.y)
        distance = np.sqrt((xDis ** 2) + (yDis ** 2))
        return distance
    
    def __repr__(self):
        return "(" + str(self.x) + "," + str(self.y) + ")"

### Fitness

The Fitness class, which represents the fitness function, possesses the properties of a path and has functions/methods used for calculating the fitness value of the path, which is based on the distance of the path. 

In [4]:
class Fitness:
    def __init__(self, route):
        self.route = route
        self.distance = None
        self.fitness = None
    
    def routeDistance(self):
        if self.distance == None:
            pathDistance = 0.0
            for i in range(0, len(self.route)):
                fromCity = self.route[i]
                toCity = None
                if i+1 < len(self.route):
                    toCity = self.route[i+1]
                else:
                    toCity = self.route[0]
                pathDistance += fromCity.distance(toCity)
            self.distance = pathDistance
        return self.distance
    
    def routeFitness(self):
        if self.fitness == None:
        # Fitness function (Simple division) that uses a simple 
        # division that divides one by the distance of the path
            self.fitness = 1 / float(self.routeDistance()) 
            # Note: You must ensure a division by zero does not occur 
        return self.fitness


## Population Initialization  

The population initialization function (or method) performs random initialization. This creates an initial population with completely random chromosomes (or solutions). There are three functions related to population initialization. 

The first function is genCityList() which generates a set of cities from a file.  

In [42]:
def genCityList(filename):
    fileCityList = []
    cityList = []
    
    # TODO 1 (10 marks) - Replace the following codes that generate 10 random cities.
    # Your new implementation must read a set of cities from the filename to be used for creating 
    # an initial population.  
    
    # Marking scheme: 
    # 7 to 10 marks:  Correct implementation. 
    # 5 to <7 marks:  Minor errors with slight effects on the fitness value.
    # >0 to <5 marks: Major errors with significant effects on the fitness value. 
    # 0 marks:        No answer is given. 
    
    # -------------- Replacement starts here  ----------------

    # Read the cities from the file
    with open(filename, 'r') as file:  
        next(file)  # Skip the header
        for line in file:
            row = line.strip().split(',')  # Split X and Y coordinates
            fileCityList.append(City(x=int(row[1]),y=int(row[2])))  # 1st column in list is ignored, 2nd and 3rd columns are x and y coordinates
    
    cityList = fileCityList

    
    # --------------- Replacement ends here -----------------
    
    return cityList


The second function is createRoute() which generates a random route (chromosome) from a set of City instances.

In [6]:
def createRoute(cityList):
    route = random.sample(cityList, len(cityList))  # len is 10, random shuffle the entire list
    return route

The third function is initialPopulation() which calls the second function repeatedly to create an initial population (a list of routes).

In [7]:
def initialPopulation(popSize, cityList):
    population = []
    for i in range(0, popSize):
        population.append(createRoute(cityList))  # Call createRoute to shuffle the list, 3 times seperately
    return population

You can run the above functions using the sample runs below. To do so, simply change the cell type from Markdown to Code.

`Sample run 1` initializes 10 cities in cityList as follows:

In [43]:
cityList = genCityList('cities50.txt') 
print(cityList)

[(34,83),
 (61,21),
 (77,16),
 (9,23),
 (56,77),
 (89,52),
 (29,58),
 (26,39),
 (75,44),
 (92,75),
 (36,1),
 (79,36),
 (1,77),
 (45,86),
 (57,30),
 (6,11),
 (97,66),
 (44,66),
 (48,32),
 (28,98),
 (76,88),
 (84,43),
 (70,32),
 (7,26),
 (95,51),
 (10,34),
 (78,28),
 (40,64),
 (39,40),
 (10,31),
 (59,3),
 (57,57),
 (100,64),
 (76,63),
 (52,35),
 (80,68),
 (98,61),
 (95,97),
 (33,60),
 (44,29),
 (15,46),
 (94,70),
 (69,80),
 (35,46),
 (14,74),
 (78,60),
 (92,57),
 (20,91),
 (15,34),
 (12,18)]


`Sample run 2` initializes 10 cities in cityList and creates a population with three routes as follows:

In [44]:
cityList = genCityList('cities50.txt') 
population = initialPopulation(3, cityList)  # 3 different routes, randomly shuffled from the list
print(population)

[[(84,43),
  (9,23),
  (98,61),
  (34,83),
  (61,21),
  (1,77),
  (95,97),
  (78,28),
  (75,44),
  (95,51),
  (92,75),
  (100,64),
  (12,18),
  (20,91),
  (76,88),
  (59,3),
  (28,98),
  (45,86),
  (10,34),
  (70,32),
  (33,60),
  (89,52),
  (39,40),
  (40,64),
  (97,66),
  (29,58),
  (26,39),
  (56,77),
  (15,46),
  (77,16),
  (52,35),
  (92,57),
  (35,46),
  (7,26),
  (80,68),
  (57,30),
  (94,70),
  (36,1),
  (79,36),
  (44,29),
  (76,63),
  (57,57),
  (6,11),
  (48,32),
  (10,31),
  (44,66),
  (14,74),
  (78,60),
  (15,34),
  (69,80)],
 [(29,58),
  (26,39),
  (78,60),
  (78,28),
  (84,43),
  (70,32),
  (57,30),
  (39,40),
  (76,63),
  (57,57),
  (69,80),
  (61,21),
  (36,1),
  (59,3),
  (95,51),
  (40,64),
  (98,61),
  (20,91),
  (89,52),
  (28,98),
  (100,64),
  (34,83),
  (10,34),
  (76,88),
  (45,86),
  (92,57),
  (92,75),
  (10,31),
  (14,74),
  (35,46),
  (77,16),
  (9,23),
  (33,60),
  (48,32),
  (94,70),
  (44,29),
  (97,66),
  (6,11),
  (95,97),
  (15,46),
  (56,77),
  (80,

## Selection

Parents selection selects chromosomes with high fitness values from a population. Survivor selection selects chromosomes with higher fitness values to form the population of the next generation. The population size is len(population), so we have len(population) in this population. 

### Parent Selection

There are three implementations for parent selection. The first parentSelection() performs **random** selection.

In [45]:
def parentSelection(population, poolSize=None):
    if poolSize == None:
        poolSize = len(population)
        
    matingPool = []
    
    for i in range(0, poolSize):
        fitness = Fitness(population[i]).routeFitness()  # Calc. fitness but not use it
        matingPool.append(random.choice(population))
      
    return matingPool

The second parentSelection() performs **Tournament** Selection.

In [63]:
def parentSelection(population, poolSize=None):
    
    # TODO 2 (10 marks) - Replace the dummy parent selection function below with  
    # Tournament Selection.
      
    # Marking scheme: 
    # 7 to 10 marks:  Correct implementation. 
    # 5 to <7 marks:  Minor errors.
    # >0 to <5 marks: Major errors. 
    # 0 marks:        No answer is given. 
    
    # You will need to compare the performance achieved by Random Selection, 
    # Tournament Selection, and Proportional Selection during performance evaluation 
    # later. So, you will run either Random Selection, Tournament Selection, or 
    # Proportional Selection in a simulation run.
    
    if poolSize == None:
        poolSize = len(population)
        
    matingPool = []
    
    # ---------------- Replacement starts here ----------------
    
    # Tournament Selection
    for i in range(0, poolSize):    
        tournament = random.sample(population, 2)  # Random pick 2 from population
        best = tournament[0]   # Assume the first one is the best
        for j in range(1, 2):  # Compare the rest with the first one, 2 used because randomly picked 2  
            if Fitness(tournament[j]).routeFitness() > Fitness(best).routeFitness():
                best = tournament[j]
        matingPool.append(best)
    
    # ---------------- Replacement ends here ----------------
    
    return matingPool

The third parentSelection() performs **Proportional** Selection.

In [12]:
def parentSelection(population, poolSize=None):
    
    # TODO 3 (10 marks) - Replace the dummy parent selection function below with  
    # Proportional Selection.
      
    # Marking scheme: 
    # 7 to 10 marks:  Correct implementation. 
    # 5 to <7 marks:  Minor errors.
    # >0 to <5 marks: Major errors. 
    # 0 marks:        No answer is given. 
    
    # You will need to compare the performance achieved by Random Selection, 
    # Tournament Selection, and Proportional Selection during performance evaluation 
    # later. So, you will run either Random Selection, Tournament Selection, or 
    # Proportional Selection in a simulation run.
    
    if poolSize == None:
        poolSize = len(population)
        
    matingPool = []
    chromosome_fitness = []
    sumfitness = 0.0
    
    # ---------------- Replacement starts here ----------------
    for i in range(0, poolSize):
        chromosome_fitness.append(Fitness(population[i]).routeFitness())
        sumfitness += Fitness(population[i]).routeFitness()

    # print("Sum fitness = " + str(sumfitness))

    j = 0
    for j in range(0, poolSize):
        choice = random.uniform(0, sumfitness)
        # print("Choice = " + str(choice))
        count = 0

        for k in range(0, poolSize):
            if count < choice:
                count += chromosome_fitness[k]
                # print("Count = " + str(count))
            
                if count >= choice:
                    matingPool.append(population[k])
                    break
    # ---------------- Replacement ends here ----------------
    
    return matingPool

### Survival Selection

In [13]:
def survivorSelection(population, eliteSize):
    
    # TODO 4 (10 marks) - Replace the dummy survival selection function below with  
    # Merge, Sort & Truncate.
      
    # Marking scheme: 
    # 7 to 10 marks:  Correct implementation. 
    # 5 to <7 marks:  Minor errors.
    # >0 to <5 marks: Major errors. 
    # 0 marks:        No answer is given. 
    
    elites = []
    
    # ---------------- Replacement starts here ----------------

    # Sort the population based on fitness score in descending order.
    sortedPopulation = sorted(population, key=lambda x: Fitness(x).routeFitness(), reverse=True)

    # sorted() = a function that returns a new sorted list
    # population = the list to be sorted
    # Fitness (x).routeFitness() =  use Fitness class with x as its input, then calculates the fitness score for the route.
    # reverse = True --> sort the list in descending order (high fitness score is the first in list)

    # Select the top eliteSize routes as elites
    elites = sortedPopulation[:eliteSize]

    # [:eliteSize] = slice the first eliceSize elements
    
    # ---------------- Replacement ends here ----------------
    
    return elites

You can run the above functions using the sample runs below. To do so, simply change the cell type from Markdown to Code. 

`Sample run 1` initializes 10 cities in cityList, creates a population with four routes, and creates a pool of parents as follows:

In [49]:
population = initialPopulation(4, genCityList('cities50.txt'))
print('Initial population') 
print(population)

matingpool = parentSelection(population, 4) 
print('Mating pool')
print(matingpool)

'Initial population'
[[(97,66),
  (44,29),
  (7,26),
  (6,11),
  (9,23),
  (80,68),
  (76,63),
  (61,21),
  (57,30),
  (69,80),
  (26,39),
  (28,98),
  (95,51),
  (14,74),
  (44,66),
  (33,60),
  (79,36),
  (1,77),
  (75,44),
  (100,64),
  (45,86),
  (48,32),
  (89,52),
  (95,97),
  (78,60),
  (78,28),
  (20,91),
  (40,64),
  (10,34),
  (15,34),
  (12,18),
  (94,70),
  (70,32),
  (10,31),
  (98,61),
  (29,58),
  (57,57),
  (92,75),
  (36,1),
  (92,57),
  (59,3),
  (76,88),
  (84,43),
  (52,35),
  (56,77),
  (39,40),
  (77,16),
  (15,46),
  (34,83),
  (35,46)],
 [(14,74),
  (10,34),
  (34,83),
  (35,46),
  (9,23),
  (75,44),
  (36,1),
  (44,66),
  (20,91),
  (28,98),
  (39,40),
  (92,75),
  (48,32),
  (94,70),
  (12,18),
  (29,58),
  (40,64),
  (80,68),
  (76,63),
  (56,77),
  (69,80),
  (1,77),
  (78,28),
  (61,21),
  (89,52),
  (57,30),
  (10,31),
  (100,64),
  (44,29),
  (97,66),
  (45,86),
  (78,60),
  (79,36),
  (33,60),
  (95,97),
  (6,11),
  (15,46),
  (7,26),
  (59,3),
  (84,43)

`Sample run 2` initializes 10 cities in cityList, creates a population with four routes, select an elite chromosome as follows:

In [50]:
population = initialPopulation(4, genCityList('cities50.txt'))
elites = survivorSelection(population, 1)
print('Initial population')
print(population)
print('Selected elites')
print(elites)

'Initial population'
[[(75,44),
  (78,60),
  (10,31),
  (12,18),
  (98,61),
  (10,34),
  (1,77),
  (44,66),
  (100,64),
  (52,35),
  (45,86),
  (95,51),
  (7,26),
  (20,91),
  (95,97),
  (33,60),
  (79,36),
  (76,63),
  (26,39),
  (44,29),
  (92,57),
  (69,80),
  (76,88),
  (9,23),
  (59,3),
  (35,46),
  (56,77),
  (6,11),
  (94,70),
  (36,1),
  (80,68),
  (57,57),
  (34,83),
  (28,98),
  (92,75),
  (70,32),
  (14,74),
  (48,32),
  (78,28),
  (57,30),
  (97,66),
  (15,34),
  (77,16),
  (61,21),
  (89,52),
  (84,43),
  (15,46),
  (40,64),
  (29,58),
  (39,40)],
 [(97,66),
  (36,1),
  (28,98),
  (79,36),
  (92,75),
  (29,58),
  (10,34),
  (78,60),
  (69,80),
  (95,97),
  (98,61),
  (40,64),
  (35,46),
  (34,83),
  (48,32),
  (92,57),
  (7,26),
  (76,63),
  (59,3),
  (70,32),
  (9,23),
  (57,30),
  (89,52),
  (15,34),
  (95,51),
  (76,88),
  (1,77),
  (52,35),
  (39,40),
  (12,18),
  (45,86),
  (56,77),
  (33,60),
  (6,11),
  (75,44),
  (94,70),
  (61,21),
  (20,91),
  (26,39),
  (78,28),

## Crossover


Crossover selects two parents, crossover the genetic materials of the parents, and produce one or more children. In the Travelling Salesman Problem, each travelling path must be valid. Each path consists all cities in the set, and each path visits each of the cities once only. So, none of the cities are visited more than once. Exchanging parts of two chromosomes tend to produce invalid paths. As an example, Parent 1 is [2 1 0 7 3 5 4 6] and Parent 2 is [6 1 0 5 2 3 4 7]. One point crossover at midpoint generates Child 1 [2 1 0 7 2 3 4 7] and Child 2 [6 1 0 5 3 5 4 6]. Both children are invalid paths.     

In [105]:
def crossover(parent1, parent2):
    
    # TODO 5 (10 marks) - Replace the dummy crossover function below with 
    # Partially Mapped Crossover approach.
   
    # Marking scheme: 
    # 7 to 10 marks:  Correct implementation. 
    # 5 to <7 marks:  Minor errors.
    # >0 to <5 marks: Major errors. 
    # 0 marks:        No answer is given. 
    
    # ---------------- Replacement starts here ----------------
    size = len(parent1)
    child1 = [None] * size
    child2 = [None] * size

    # Random select two cut points, ensure two are different
    cross_point1 = random.randint(0, size)
    cross_point2 = random.randint(0, size)

    while cross_point1 == cross_point2:
        cross_point2 = random.randint(0, size)
    
    start, end = min(cross_point1, cross_point2), max(cross_point1, cross_point2)

    # Select from parent
    child1[start:end] = parent1[start:end]
    child2[start:end] = parent2[start:end]

    # Fill the rest of the child with the remaining genes from the other parent
    child1_set = set(child1)  # Use set for faster lookup
    child2_set = set(child2)

    for i in range(size):
        if i < start or i >= end:
            for j in range(size):
                if parent2[j] not in child1_set:
                    child1[i] = parent2[j]
                    child1_set.add(parent2[j])
                    break
            for j in range(size):
                if parent1[j] not in child2_set:
                    child2[i] = parent1[j]
                    child2_set.add(parent1[j])
                    break

    # ---------------- Replacement ends here ----------------
    
    return child1, child2

Crossover selects two parents from the mating pool to produce a new generation of the same size.

In [52]:
def breedPopulation(matingpool):
    children = []
    
    # Choosing parents in their order of presence in the mating pool. Choosing parents
    # in a random manner is possible. 
    
    for i in range(1, len(matingpool), 2):
        child1, child2 = crossover(matingpool[i-1], matingpool[i])
        children.append(child1)
        children.append(child2)
    
    return children

You can run the above functions using the sample run below. To do so, simply change the cell type from Markdown to Code. The sample run initializes 2 chromosomes in the population, and performs crossover among the two parents. 

In [106]:
population = initialPopulation(2, genCityList('cities500.txt'))
parent1, parent2 = population
child1, child2 = crossover(parent1, parent2)
print('Parents')
print(parent1)
print(parent2)
print('Children')
print(child1)
print(child2)

'Parents'
[(78,94),
 (6,11),
 (67,91),
 (85,5),
 (53,79),
 (56,51),
 (61,21),
 (55,3),
 (53,88),
 (35,72),
 (69,78),
 (3,33),
 (75,54),
 (40,64),
 (60,70),
 (63,62),
 (4,72),
 (3,9),
 (4,21),
 (70,70),
 (27,81),
 (88,38),
 (95,51),
 (26,39),
 (56,5),
 (51,78),
 (63,58),
 (57,71),
 (93,85),
 (1,77),
 (28,27),
 (67,4),
 (91,13),
 (83,21),
 (57,81),
 (36,97),
 (22,73),
 (63,49),
 (44,4),
 (55,15),
 (76,79),
 (79,36),
 (44,54),
 (58,2),
 (35,67),
 (58,38),
 (17,39),
 (84,30),
 (70,41),
 (39,49),
 (4,24),
 (35,31),
 (89,69),
 (88,17),
 (35,46),
 (93,30),
 (76,49),
 (56,27),
 (60,60),
 (48,35),
 (74,98),
 (15,92),
 (73,68),
 (25,80),
 (69,2),
 (76,25),
 (12,18),
 (27,86),
 (10,69),
 (76,88),
 (15,51),
 (30,20),
 (84,0),
 (5,44),
 (27,17),
 (13,87),
 (71,27),
 (12,56),
 (60,35),
 (44,64),
 (15,70),
 (57,57),
 (41,97),
 (0,29),
 (89,31),
 (51,32),
 (46,57),
 (11,16),
 (54,65),
 (39,46),
 (41,38),
 (2,61),
 (58,21),
 (69,80),
 (84,5),
 (1,56),
 (90,2),
 (66,100),
 (83,44),
 (56,80),
 (68,1),
 (

## Mutation

Mutation mutates a single chromosome to get a mutated chromosome so that genetic algorithm can converge to a shorter path quickly. In the Travelling Saleman Problem, a mutated chromosome must be a valid path. As an example, the insertion mutation randomly inserts a single gene in the [1 2 3 4 5 6 7 8 9 10] chromosome to generate the [1 2 4 5 6 7 3 8 9 10] mutated chromosome. Step 1: select a gene randomly, Step 2: insert this gene into a randomly selected location.

In [55]:
def mutate(route, mutationProbability):
    
    # TODO 6 (10 marks) - Replace the dummy mutation function below with Insertion Mutation.
    # The dummy mutation function simply swaps a city with the city before it.  
   
    # Marking scheme: 
    # 7 to 10 marks:  Correct implementation. 
    # 5 to <7 marks:  Minor errors.
    # >0 to <5 marks: Major errors. 
    # 0 marks:        No answer is given. 
     
    mutated_route = route[:]
    for i in range(len(route)):
        if (random.random() < mutationProbability):
            # mutationProbability is the probability of a gene undergoing mutation
            
            # ---------------- Replacement starts here ----------------
            # print(i)
            city = mutated_route.pop(i)
            insert_point = random.randint(0, len(mutated_route))
            mutated_route.insert(insert_point, city)

            # print(mutated_route)
            # ---------------- Replacement ends here ----------------
    return mutated_route

Mutation runs over the entire population and mutates each chromosome in the population with a small mutationProbability. 

In [56]:
def mutation(population, mutationProbability):
    mutatedPopulation = []
    for i in range(0, len(population)):
        mutatedIndividual = mutate(population[i], mutationProbability)
        mutatedPopulation.append(mutatedIndividual)
    return mutatedPopulation

You can run the above functions using the sample run below. To do so, simply change the cell type from Markdown to Code. The sample run initializes a route comprised of 10 cities in cityList, and then mutates it as follows:

In [59]:
route = genCityList('cities50.txt')
mutated = mutate(route, 1)  # Give a pretty high chance for mutation
print('Original route')
print(route)
print('Mutated route')
print(mutated)

'Original route'
[(34,83),
 (61,21),
 (77,16),
 (9,23),
 (56,77),
 (89,52),
 (29,58),
 (26,39),
 (75,44),
 (92,75),
 (36,1),
 (79,36),
 (1,77),
 (45,86),
 (57,30),
 (6,11),
 (97,66),
 (44,66),
 (48,32),
 (28,98),
 (76,88),
 (84,43),
 (70,32),
 (7,26),
 (95,51),
 (10,34),
 (78,28),
 (40,64),
 (39,40),
 (10,31),
 (59,3),
 (57,57),
 (100,64),
 (76,63),
 (52,35),
 (80,68),
 (98,61),
 (95,97),
 (33,60),
 (44,29),
 (15,46),
 (94,70),
 (69,80),
 (35,46),
 (14,74),
 (78,60),
 (92,57),
 (20,91),
 (15,34),
 (12,18)]
'Mutated route'
[(56,77),
 (61,21),
 (9,23),
 (98,61),
 (35,46),
 (89,52),
 (26,39),
 (80,68),
 (44,29),
 (57,30),
 (92,75),
 (79,36),
 (1,77),
 (97,66),
 (29,58),
 (45,86),
 (78,60),
 (95,51),
 (15,34),
 (10,31),
 (20,91),
 (75,44),
 (57,57),
 (6,11),
 (14,74),
 (100,64),
 (44,66),
 (92,57),
 (28,98),
 (12,18),
 (84,43),
 (39,40),
 (7,26),
 (76,88),
 (77,16),
 (48,32),
 (34,83),
 (40,64),
 (59,3),
 (78,28),
 (10,34),
 (52,35),
 (76,63),
 (69,80),
 (70,32),
 (95,97),
 (33,60),
 (36,1

## Running One Generation (or Interation)

Here, we run one generation of genetic algorithm. 

In [60]:
def oneGeneration(population, eliteSize, mutationProbability):
    
    # First we preserve the elites
    elites = survivorSelection(population, eliteSize)
    
    # Then we calculate what our mating pool size should be and generate
    # the mating pool
    poolSize = len(population) - eliteSize
    matingpool = parentSelection(population, poolSize)
        
    # Then we perform crossover on the mating pool
    children = breedPopulation(matingpool)
    
    # We combine the elites and children into one population
    new_population = elites + children
    
    # We mutate the population
    mutated_population = mutation(new_population, mutationProbability)
        
    return mutated_population

You can run the above functions using the sample run below. To do so, simply change the cell type from Markdown to Code. The sample run initializes a population comprised of 5 chromosomes based on 10 cities in cityList, and then run one generation (or iteration) of genetic algorithm as follows:

In [61]:
population = initialPopulation(5, genCityList('cities50.txt'))
eliteSize = 1
mutationProbability = 0.01
new_population = oneGeneration(population, eliteSize, mutationProbability)
print('Initial population')
print(population)
print('New population')
print(new_population)

'Initial population'
[[(94,70),
  (10,31),
  (52,35),
  (45,86),
  (33,60),
  (28,98),
  (84,43),
  (95,97),
  (59,3),
  (97,66),
  (7,26),
  (10,34),
  (48,32),
  (76,63),
  (6,11),
  (98,61),
  (44,66),
  (92,57),
  (20,91),
  (79,36),
  (95,51),
  (78,60),
  (76,88),
  (56,77),
  (69,80),
  (61,21),
  (77,16),
  (92,75),
  (57,30),
  (34,83),
  (12,18),
  (35,46),
  (57,57),
  (9,23),
  (15,46),
  (70,32),
  (44,29),
  (26,39),
  (75,44),
  (29,58),
  (80,68),
  (14,74),
  (78,28),
  (36,1),
  (100,64),
  (15,34),
  (89,52),
  (40,64),
  (39,40),
  (1,77)],
 [(76,88),
  (35,46),
  (95,51),
  (98,61),
  (12,18),
  (36,1),
  (77,16),
  (20,91),
  (52,35),
  (97,66),
  (45,86),
  (78,60),
  (70,32),
  (26,39),
  (15,34),
  (92,75),
  (44,29),
  (95,97),
  (57,57),
  (48,32),
  (7,26),
  (94,70),
  (10,34),
  (40,64),
  (79,36),
  (92,57),
  (10,31),
  (15,46),
  (89,52),
  (61,21),
  (59,3),
  (75,44),
  (28,98),
  (84,43),
  (34,83),
  (14,74),
  (6,11),
  (44,66),
  (57,30),
  (76,63

## Running Many Generations (or Interations) 

In [108]:
filename = 'cities500.txt'
popSize = 20
eliteSize = 5
mutationProbability = 0.01
iteration_limit = 100

cityList = genCityList(filename)

population = initialPopulation(popSize, cityList)
distances = [Fitness(p).routeDistance() for p in population]
min_dist = min(distances)
print("Best distance for initial population: " + str(min_dist))

for i in range(iteration_limit):
    population = oneGeneration(population, eliteSize, mutationProbability)
    distances = [Fitness(p).routeDistance() for p in population]
    index = np.argmin(distances)
    best_route = population[index]
    min_dist = min(distances)
    print("Best distance for population in iteration " + str(i) +
          ": " + str(min_dist))

print("Optimal path is " + str(best_route)) 

    # TODO 7 (10 marks) - Performance Evaluation. You will present the performance achieved 
    # by different parent selection function. You will compare the 
    # performance achieved by Random Selection, Tournament Selection, and Proportional Selection. 
   
    # Marking scheme: 
    # 7 to 10 marks:  In-depth performance evaluation. Optimal routes are found. 
    # 5 to <7 marks:  Clear understanding of performance evaluation.
    # >0 to <5 marks: Inaccurate or unclear understanding of performance evaluation. 
    # 0 marks:        No answer is given. 
    
  

'Best distance for initial population: 25568.805147099607'
'Best distance for population in iteration 0: 25282.7197618747'
'Best distance for population in iteration 1: 25285.75153388737'
'Best distance for population in iteration 2: 24966.724945330232'
'Best distance for population in iteration 3: 24766.10770879836'
'Best distance for population in iteration 4: 24772.367158695728'
'Best distance for population in iteration 5: 24495.089810260135'
'Best distance for population in iteration 6: 24364.16130289439'
'Best distance for population in iteration 7: 24473.018750147687'
'Best distance for population in iteration 8: 24130.49864073939'
'Best distance for population in iteration 9: 24042.49141138289'
'Best distance for population in iteration 10: 23927.138242048688'
'Best distance for population in iteration 11: 23707.05618839304'
'Best distance for population in iteration 12: 23921.80047561676'
'Best distance for population in iteration 13: 23791.087889763945'
'Best distance for pop